# Multiple tests for Solving Kyber_256

In [1]:
from lwe_with_hints import *

In [2]:
# lattice.s

In [3]:
# lattice.shortestVector

In [4]:
# lattice.successBlocksize

In [5]:
# lattice = LWELattice(A,b,q)

## Simulating Kyber 256

In [6]:
ntt_n = 128
KYBER_K = 2

n = KYBER_K *ntt_n
m = n
q = 3329

instance  = "Kyber"+str(n)

In [7]:
def experiment(A,b,q,hints,modular,fileName,verbose ):
  
  lattice = LWELattice(A,b,q,verbose=verbose)
  V, L = hints
  ctrHints = len(V)
  
  try:
    start = time.time()
    
    for i in range(ctrHints):
      if modular:
        lattice.integrateModularHint( V[i], L[i] % q, q )
      else:
        lattice.integratePerfectHint( V[i], L[i] )
    
    lattice.reduce(maxBlocksize=40)
    stop = time.time()
    
    output = "Finished experiment.\tHints: %d\tBlocksize: %d\tTime: %fs" % (ctrHints, lattice.successBlocksize, (stop-start))
    print( "\033[94m" + output + "\033[0m" )
    print(lattice.s)
    with open(fileName, "a+") as f:
      print(output, file=f)
      
        
  except Exception:
    #Print exceptions manualy, because Pool may hide them.
    output = traceback.format_exc()
    print(output)
    with open(fileName, "a+") as f:
      print(output, file=f)

In [8]:
R = IntegerModRing(3329)
V = VectorSpace(R,ntt_n)

#incomplete ntt
if ntt_n == 128:
    ntt_layer = 6
    zeta = 17^2
elif ntt_n == 64:
    ntt_layer = 5
    zeta = 17^4


import numpy as np

def bit_reverse(x):return 2*int( "0b" + bin(x)[2:].rjust(ntt_layer,'0')[::-1] ,2)+1

NTT_matrix = []

def add (x,y) : return x +y

for x in range(ntt_n/2):
    NTT_matrix.append(V(reduce(add, [[(zeta)^(x*bit_reverse(i)),0] for i in range(ntt_n/2)])))
    NTT_matrix.append(V(reduce(add, [[0,(zeta)^(x*bit_reverse(i))] for i in range(ntt_n/2)])))

NTT_matrix = matrix(NTT_matrix)

inv_NTT_matrix = NTT_matrix^-1

## Multiple TEST

In [9]:
from multiprocessing import Pool
import time
import traceback
def generateHints(s):
    s_list = [s[ntt_n*i:ntt_n*(i+1)] for i in range(KYBER_K)]
    s_hat_list = [V(list(i))*NTT_matrix for i in s_list]
    # Hint_list
    s_relation_list = [ [R(s_hat[2*i])/R(s_hat[2*i+1]) for i in range(ntt_n/2)]  for s_hat in s_hat_list]
    
    v_list = []
    v_list_1 = []
    for j in range(KYBER_K):
        for i in range(ntt_n/2):
            s_relation = s_relation_list[j]
            k = s_relation[i]
            v = list((NTT_matrix.column(2*i) - k*NTT_matrix.column(2*i+1)))
            v = [0]*int(ntt_n*j) + v + [0]*int(ntt_n*(KYBER_K-j-1))
            v_prime  = [int(i) for i in list(v) + [0]*(m)]
        
            v_prime= vec(v_prime)
            v_list.append(v_prime)
            v_list_1.append(v)
            # print(dbdd.leak(v_prime)%3329)

    for v0 in v_list_1:
        mul_res = sum([v0[i]* s[i] for i in range(len(v0))])%q
    
    if mul_res!=0:
        assert("hint is wrong")

    return  (v_list_1,[0]*int(n/2))

In [10]:
#Test for 8 example in 8 cores
trials=8
modular = True
verbose = True
fileName ='Multiple_thread_output.txt'
scheme = 'Kyber256'
instances=[]

for i in range(trials):
  
  A,b,q,s,e = generateLWEInstance( scheme )
  
  print(s)
  
  hints = generateHints(s)

  instances.append( (A, b, q, hints, modular, fileName,verbose) )

pool = Pool()
pool.starmap( experiment, instances )

[ 0  1 -1  0 -1  0  0 -1  0  1 -1  0  1  1  1  1  0  0 -1  0 -1 -1  0  0
 -1  2  0  0  0 -1 -2 -1  0 -1  0 -1 -1  0  0  0  0  1  1  0 -1  2  0 -1
 -2  1  1  2 -1  1  0 -1  0 -2  0 -2  1  2  0  0 -1 -1 -2 -1  1 -1 -1  0
  0 -1 -1  0  1  0  0  1  0 -1  0 -1 -2  0 -2  1 -1  1  0  0 -2 -1  1 -1
 -2  1 -1  1  2  0 -2  1 -2  0 -1  1  0 -1  1 -1 -1  0  0  2  2  2  0  0
 -2  1  2  1  0  1  1 -2  0  0 -1  0  0  0 -1  2  0  1 -2 -1  0  0  0  0
  1  0  1  0  1  1  2  1  0 -1  0  2 -1 -1  0 -1 -1  0  2 -1 -1  1  0  1
  0  0 -1 -1 -2 -1  0 -1  1 -1  0  0  0  1  1 -1  1  1 -2  1  1  0  0  0
 -1 -2  2  1  0  1 -1  2  0 -1  1  0 -1  2  0  0  0 -1  1 -1 -2  0  1  1
  0 -1 -1  0  1  0  2  0 -1  1 -2  0 -1  1  0  1  0  0  2 -1  0  0  1  0
  1 -1 -1 -2  1  1  0  1  2 -2  0  0  1  0  0  0]
[-1  1  0  1 -2 -1 -1 -1 -1  0  0  0  1  0  1 -1 -1  0  0  0  0 -2  0  1
  2  0  2  1 -1 -1  0  0  1  1  0  0  0  1  1 -1  0 -1 -2 -1  2  0 -1  0
  1  0 -1  2 -1 -1 -1 -1 -1  0  1  0  0  0 -1  0  0 -1  0  2 -2 -2 -2  1
 

[None, None, None, None, None, None, None, None]